In [4]:

import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
from tqdm.auto import tqdm
from glob import glob
from shapely.geometry import LineString, Point
import folium
pd.set_option('display.max_columns', None)

In [6]:
new_shoreline = gpd.GeoDataFrame.from_features({"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[8.853934,38.87871],[8.854262,38.878702],[8.854535,38.878635],[8.854771,38.878568],[8.855023,38.878481],[8.855093,38.878376]]}}]}, crs="EPSG:4326")
new_shoreline

,geometry
0,"LINESTRING (8.85393 38.87871, 8.85426 38.8787,..."


In [9]:
new_shoreline.explore(tiles="ESRI.WorldImagery")

In [10]:
shorelines = gpd.read_file("shorelines.geojson")
transects = gpd.read_file("transects_extended.geojson")
poly = gpd.read_file("polygons.geojson")

In [11]:
sar_shorelines = shorelines[shorelines.id.str.startswith("sar")]
latest_siteid = sar_shorelines.id.max()
new_siteid = f"sar{int(latest_siteid[3:])+1}"
print(f"Latest siteid is {latest_siteid}, so this new site will be {new_siteid}")
new_shoreline["id"] = new_siteid
new_shoreline

Latest siteid is sar2540, so this new site will be sar2541


,geometry,id
0,"LINESTRING (8.85393 38.87871, 8.85426 38.8787,...",sar2541


In [12]:
pd.concat((shorelines, new_shoreline)).to_file("shorelines.geojson", driver="GeoJSON")

In [14]:
new_poly = gpd.GeoDataFrame([{"id": new_siteid}], geometry=new_shoreline.to_crs(32632).buffer(100).minimum_rotated_rectangle().to_crs(4326), crs=4326)
new_poly.explore()

In [15]:
pd.concat((poly, new_poly)).to_file("polygons.geojson", driver="GeoJSON")

In [16]:
sar_transects = transects[transects.id.str.startswith("sar")].to_crs(32632)
sar_transects

,id,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
127429,sar0001-0000,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,0.068364,664.0,508.0,0.000113,52.578021,6137.692249,78.343425,124.161754,Medium,"LINESTRING (448682.948 4301601.333, 448711.634..."
127430,sar0001-0001,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.197266,664.0,498.0,0.002798,19.969642,2010.059995,44.833693,197.183041,Medium,"LINESTRING (448672.359 4301586.908, 448893.248..."
127431,sar0001-0002,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.009191,664.0,545.0,0.000059,11.768691,215.362959,14.675250,206.410997,Medium,"LINESTRING (448718.143 4301576.257, 449018.083..."
127432,sar0001-0003,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.040081,664.0,597.0,0.002059,8.475977,112.442187,10.603876,239.306868,Medium,"LINESTRING (448727.641 4301549.522, 449101.288..."
127433,sar0001-0004,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,0.009671,664.0,603.0,0.000141,7.011139,96.998364,9.848775,258.388646,Medium,"LINESTRING (448728.444 4301602.089, 449096.69 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147182,sar2540-0000,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,0.230656,1261.0,1068.0,0.014379,19.726461,544.099943,23.325950,208.319661,High,"LINESTRING (480984.841 4307314.15, 480852.713 ..."
147183,sar2540-0001,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,-0.179234,1261.0,1218.0,0.042395,8.166520,107.185533,10.353045,179.467747,High,"LINESTRING (481064.051 4307268.819, 480863.194..."
147184,sar2540-0002,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,-0.182292,1261.0,1226.0,0.045167,8.070813,103.598035,10.178312,174.938721,High,"LINESTRING (481101.193 4307235.355, 480900.346..."
147185,sar2540-0003,sar2540,NaN,NaN,NaN,NaN,NaN,NaN,-0.156003,1261.0,1204.0,0.037256,7.475358,92.953779,9.641254,169.587577,High,"LINESTRING (481157.467 4307167.599, 480906.533..."


In [17]:
def create_transects(line, spacing=25, transect_length=300):
    transects = []
    distances = np.arange(0, line.length, spacing)

    for distance in distances:
        # Find point along the average line
        point = line.interpolate(distance)
        
        # Find the direction of the line at this point (tangent direction)
        nearest_point_ahead = line.interpolate(min(distance + 1e-6, line.length))
        direction = np.arctan2(nearest_point_ahead.y - point.y, nearest_point_ahead.x - point.x)
        
        # Rotate 90 degrees (perpendicular) and extend to create a transect
        transect = LineString(reversed([
            Point(
                point.x - transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y - transect_length / 2 * np.sin(direction + np.pi / 2)
            ),
            Point(
                point.x + transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y + transect_length / 2 * np.sin(direction + np.pi / 2)
            )
        ]))
        
        transects.append(transect)
    
    transects = gpd.GeoDataFrame(geometry=transects, crs=32632).to_crs(4326)
    return transects

new_transects = create_transects(new_shoreline.to_crs(32632).geometry.iloc[0])
display(new_transects)
m = new_transects.explore()
new_shoreline.explore(m=m)
new_poly.boundary.explore(m=m)
gpd.GeoSeries(new_transects.geometry.apply(lambda line: Point(line.coords[0])), crs=new_transects.crs).explore(m=m, color="red", name="transect start")
print("Make sure the origin is inland")
m

,geometry
0,"LINESTRING (8.85399 38.88006, 8.85388 38.87736)"
1,"LINESTRING (8.85428 38.88005, 8.85417 38.87735)"
2,"LINESTRING (8.85502 38.87993, 8.85398 38.87735)"
3,"LINESTRING (8.85536 38.87984, 8.85418 38.8773)"
4,"LINESTRING (8.85656 38.8791, 8.85349 38.87785)"


Make sure the origin is inland


In [18]:
new_transects["id"] = new_siteid + "-" + new_transects.index.astype(str).str.pad(4, fillchar="0")
new_transects["site_id"] = new_siteid
#new_transects["beach_slope"] = .1
new_transects

,geometry,id,site_id
0,"LINESTRING (8.85399 38.88006, 8.85388 38.87736)",sar2541-0000,sar2541
1,"LINESTRING (8.85428 38.88005, 8.85417 38.87735)",sar2541-0001,sar2541
2,"LINESTRING (8.85502 38.87993, 8.85398 38.87735)",sar2541-0002,sar2541
3,"LINESTRING (8.85536 38.87984, 8.85418 38.8773)",sar2541-0003,sar2541
4,"LINESTRING (8.85656 38.8791, 8.85349 38.87785)",sar2541-0004,sar2541


In [19]:
pd.concat((transects, new_transects)).to_file("transects_extended.geojson", driver="GeoJSON")